In [2]:
import numpy as np

In [3]:
# с семинара

num_companies = 4  # кол-во компаний
capital = 100  # в тыс. руб.
step = 10
profit_of_investment = [[0, 7, 13, 20, 33, 35],
                        [0, 8, 12, 21, 28, 40],
                        [0, 10, 11, 19, 30, 34],
                        [0, 6, 9, 24, 30, 36]]

In [4]:
# мой вариант

num_companies = 4  # кол-во компаний
capital = 700  # в тыс. руб.
step = 100
profit_of_investment = [[0, 28, 45, 65, 78, 90, 102, 113],
                        [0, 25, 41, 55, 65, 75, 80, 85],
                        [0, 15, 25, 40, 50, 62, 73, 82],
                        [0, 20, 33, 42, 48, 53, 56, 58]]

In [13]:
# my test case

num_companies = 3  # кол-во компаний
capital = 50  # в тыс. руб.
step = 10
profit_of_investment = [[0, 1],
                        [0, 1, 2, 3],
                        [0, 1, 2]]

In [6]:
# Никита

capital = 700
step = 100
num_companies = 4
profit_of_investment = [
    [0, 3, 18, 59, 66, 64, 72, 81],
    [0, 8, 24, 26, 40, 42, 51, 57],
    [0, 6, 15, 17, 31, 33, 45, 67],
    [0, 9, 30, 45, 67, 70, 72, 74],
]

In [33]:
def calc_possible_u(profit_of_investment, step):
    """
    Считаем возможные управления U_matrix. k-строка соответствует k-ому шагу.
    :param profit_of_investment: лист из листов с доходом от предприятий
    :param step: шаг с которым меняются управления
    :return: лист из листов возможных управлений
    """
    U_matrix = []
    for k in range(0, len(profit_of_investment)):  # идем по предприятиям
        max_u_index = len(profit_of_investment[k])
        U = [i * step for i in range(max_u_index)] # заполняем строку матрицы
        U_matrix.append(U) # добавляем строку в матрицу
    return U_matrix

def calc_possible_x(profit_of_investment, step):
    """
    Считаем возможные состояния X_matrix. k-строка соответствует k-ому шагу.
    :param profit_of_investment: лист из листов с доходом от предприятий
    :param step: шаг с которым меняются управления
    :return:
    """
    X_matrix = []
    U_matrix = calc_possible_u(profit_of_investment, step)
    x_initial = capital
    for k in range(0, len(profit_of_investment)):  # идем по предприятиям
        # посчитаем все возможные состояния после того, как подействуем управлением u на k шаге на состояние x после k-1 шага
        X = []
        if k == 0:
            for u in U_matrix[k]:
                    X.append(x_initial - u)
        else:
            for x in X_matrix[k - 1]:
                for u in U_matrix[k]:
                    X.append(x - u)
        X = list(set(X))  # удалим дубликаты
        X = sorted(X)  # отсортируем
        X = [x for x in X if x >= 0]  # уберем отрицательные значения
        X_matrix.append(X)
    return X_matrix

In [35]:
# итоговая функция

def find_maximum_profit(profit_matrix, capital):
    U_matrix = calc_possible_u(profit_matrix, step)
    X_matrix = calc_possible_x(profit_matrix, step)
    num_companies = len(profit_matrix)
    F = {} # хеш-таблицу, в которой мы храним все значения функции Беллмана
    for k in range(num_companies - 1, -1, -1):  # идем от (num_companies - 1) до 0
        if k == 0:  # заполняем лист возможных x (состояний)
            X = [capital]
        else:
            X = [i for i in X_matrix[k - 1]]
        U = [i for i in U_matrix[k]]  # заполняем лист возможных u (управлений)
        for x in X:

            # перебираем все возможные управления
            w_list = []
            for u in U:
                if u > x:  # если наше управление предполагает страты большие, чем у нас есть денег
                    break  # то значит такое управление невалидно => выходим из цикла
                if k == num_companies - 1:  # если мы на шаге, соответсвующему последней компании
                    w_list.append([profit_matrix[k][U.index(u)], u])  # сохраняем и значение F, и управление
                else:
                    w_list.append([profit_matrix[k][U.index(u)] + F[(k + 1, x - u)]["w"], u])

            # ищем максимальную прибыль
            w_max = 0  # максимальная прибыль
            for w, u in w_list:
                if w > w_max:
                    w_max = w
            u_max_list = []  # управления, при которых у нас достигается максимальная прибыль
            for w, u in w_list:  # сохраняем все возможные управления, соответствующие максимальное прибыли
                if w == w_max:
                    u_max_list.append(u)
            F[(k, x)] = {
                "w": w_max,
                "u": u_max_list
            }
            print("F({0}, {1}) = ".format(k, x), F[(k, x)])

    print("Answer = ", F[(0, capital)])
    print()

    paths = [{
        "x": capital,
        "u": []
    }]
    for k in range(0, num_companies):
        new_paths = []
        for path in paths:
            x_of_path = path["x"]
            u_of_path = path["u"]
            U = F[(k, x_of_path)]["u"]
            for u in U:
                new_paths.append({
                    "x": x_of_path - u,
                    "u": [*u_of_path, u]
                })
        paths = new_paths
    paths = [path["u"] for path in paths]
    print(paths)

    for i in range(1):
        # print("Variant 1:")
        pass
        for j in range(1):
            # print("It should be spent (*amount1*) on company (*number1*)")
            # print("Total profit is (*target value*))")
            pass


find_maximum_profit(profit_of_investment, capital)

F(2, 10) =  {'w': 1, 'u': [10]}
F(2, 20) =  {'w': 2, 'u': [20]}
F(2, 30) =  {'w': 2, 'u': [20]}
F(2, 40) =  {'w': 2, 'u': [20]}
F(2, 50) =  {'w': 2, 'u': [20]}
F(1, 40) =  {'w': 4, 'u': [20, 30]}
F(1, 50) =  {'w': 5, 'u': [30]}
F(0, 50) =  {'w': 5, 'u': [0, 10]}
Answer =  {'w': 5, 'u': [0, 10]}

[[0, 30, 20], [10, 20, 20], [10, 30, 10]]
